In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
data = pd.read_csv("../Data/vix/TRBContest_Deliverables/trip_leg_matrix_2014_04_03_DDP.csv")
print data.shape

(180874, 9)


In [3]:
print data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180874 entries, 0 to 180873
Data columns (total 9 columns):
Origin_Zone         180874 non-null int64
Destination_Zone    180874 non-null int64
Start_Date          180874 non-null int64
End_Date            180874 non-null int64
Aggregation         180874 non-null object
Subscriber_Class    180874 non-null object
Purpose             180874 non-null object
Time_of_Day         180874 non-null object
Count               180874 non-null float64
dtypes: float64(1), int64(4), object(4)
memory usage: 12.4+ MB
None


In [4]:
print data.describe()

        Origin_Zone  Destination_Zone  Start_Date    End_Date          Count
count  1.808740e+05      1.808740e+05    180874.0    180874.0  180874.000000
mean   9.904132e+13      1.014076e+14  20140403.0  20140403.0      81.364012
std    2.987183e+14      3.018685e+14         0.0         0.0     652.837734
min    1.000000e+00      1.000000e+00  20140403.0  20140403.0       3.000000
25%    4.930000e+02      4.930000e+02  20140403.0  20140403.0      28.370000
50%    8.350000e+02      8.350000e+02  20140403.0  20140403.0      38.980000
75%    1.067000e+03      1.070000e+03  20140403.0  20140403.0      52.870000
max    1.000000e+15      1.000000e+15  20140403.0  20140403.0   32792.670000


In [5]:
ordes = data[["Origin_Zone","Destination_Zone"]]

ordes.ix[ordes.Origin_Zone > 2000] %= 5000 
ordes.ix[ordes.Destination_Zone > 2000] %= 5000
print ordes.head()

   Origin_Zone  Destination_Zone
0         4005              4005
1         4005              4005
2         4000              4000
3         4001              4001
4          973               985


/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
time_division = data[["Time_of_Day"]]
#print time_division.head()

def start_cleaner(value):
    """
    Clean and divide into start and end time
    """
    def start_time(val):
        return int(val[1:3])
    
    start = value.apply(start_time)
    return start

def end_cleaner(value):
    def end_time(val):
        return int(val[5:]) 
    end = value.apply(end_time)
    return end
        
x = time_division.apply(start_cleaner)
y = time_division.apply(end_cleaner)
z = pd.concat([x,y],axis = 1)
z.columns = ["Start_time","End_time"]
print z.head()

   Start_time  End_time
0           0         1
1           0         1
2           0         1
3           4         5
4           0         1


In [7]:
date = data.Start_Date.apply(lambda x : x%100)
print date.head()

0    3
1    3
2    3
3    3
4    3
Name: Start_Date, dtype: int64


In [8]:
data = data[["Aggregation","Subscriber_Class","Purpose","Count"]]
print data.head()

  Aggregation    Subscriber_Class Purpose    Count
0         Thu  Short Term Visitor      OH  3948.76
1         Thu  Short Term Visitor      HH  3472.69
2         Thu    Inbound Commuter      OH   244.23
3         Thu  Short Term Visitor      OH   560.45
4         Thu   Long Term Visitor      OO    30.39


In [9]:
final_data = pd.concat([ordes,date,data,z],axis = 1)
print final_data.head()

   Origin_Zone  Destination_Zone  Start_Date Aggregation    Subscriber_Class  \
0         4005              4005           3         Thu  Short Term Visitor   
1         4005              4005           3         Thu  Short Term Visitor   
2         4000              4000           3         Thu    Inbound Commuter   
3         4001              4001           3         Thu  Short Term Visitor   
4          973               985           3         Thu   Long Term Visitor   

  Purpose    Count  Start_time  End_time  
0      OH  3948.76           0         1  
1      HH  3472.69           0         1  
2      OH   244.23           0         1  
3      OH   560.45           4         5  
4      OO    30.39           0         1  


In [10]:
final_data.to_csv("matrix_3.csv",index = False)